In [2]:
import sympy as sp
import numpy as np
from IPython.display import display, Math
import matplotlib.pyplot as plt

z = sp.Symbol("z", complex = True)
n, k, j = sp.symbols("n k j", integer = True)
N = 20

In [3]:
# Insertar phi y theta
phi = 1 - 0.1 * z - 0.12 * z**2
theta = 1 - 7/10 * z
sigma = 1 # standard deviation
X = [0, .644, -.442, -.919, -1.573, - 0.852, -.907, .686, -.753, -.954, .576] # X_0 = 0

In [4]:
# phi = 1 - z + 0.24*z**2
# theta = 1 + 0.4*z + 0.2*z**2 + 0.1*z**3
# sigma = 1
# X = [0, 1.704, 0.527, 1.041, 0.942, 0.555, -1.002, -0.585, 0.010, -0.638, 0.525] # X_0 = 0

In [5]:
phi = sp.Poly(phi)
theta = sp.Poly(theta)
p = phi.degree()
q = theta.degree()
m = max(p,q)
center = max(p,q+1)


phi_coeff = lambda k: -phi.coeffs()[-1-k] if 0< k <= p else 1 if k == 0 else 0
theta_coeff = lambda k: theta.coeffs()[-1-k] if 0< k <= q else 1 if k == 0 else 0

In [6]:
# Calculo de psi(z) y expansion en serie de potencias


partial_fraction_expansion = sp.apart(theta/phi, full=True).nsimplify(tolerance=1e-10)
display(Math("\\psi(z) =" + sp.latex(partial_fraction_expansion)))
psi_n = 0
polynomials = []
for partial_fraction in partial_fraction_expansion.args:
    if partial_fraction.is_polynomial() :
        polynomials.append(partial_fraction)
    else:
        poly = sp.Poly(partial_fraction**(-1), z)
        coeffs = (poly).coeffs()
        degree = sp.degree(poly, z)
        a = coeffs[-1]**(-1)
        r = sp.root((a*(-1)**degree) * coeffs[0],degree)
        psi_n += a*sp.binomial(n,degree-1)*r**n

polynomial_part = []
try:
    polynomial_part = sp.Poly(sum(polynomials)).coeffs()[::-1]
except:
    None

def psi(i):
    return (psi_n + sum([sp.Piecewise((polynomial_part[l], sp.Eq(n,l)), (0, True)) for l in range(len(polynomial_part))])).subs({n:i})
display(Math("\\psi_n =" + sp.latex(psi(n).simplify())))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
# Calculo de gamma
gamma_expr = 0
for expr in ((psi(n) * psi(n+k)).expand()).args:
    log_expr = sp.expand_log(sp.log(expr), force = True).collect(k).collect(n)
    final_expr = 1
    for expr1 in log_expr.args:
        final_expr *= sp.powdenest(sp.exp(expr1.simplify()), force = True)
    gamma_expr += sp.Sum( final_expr,  (n,0,sp.oo))
gamma_expr = ((sp.Integer(0) + gamma_expr).doit()).simplify()

display(Math("\\gamma(k) =" + sp.latex(gamma_expr)))
# gamma = lambda x: gamma_expr.subs({k:x}).simplify()
gamma_1 = sp.lambdify(k,gamma_expr)
gamma = lambda k: gamma_1(abs(k))

<IPython.core.display.Math object>

In [8]:
def get_autocovariance(phi, theta,N = N): 
    psi = [theta_coeff(0)]
    for j in range(1, max(p,q+1)):
        psi_j = theta_coeff(j) + sum([phi_coeff(k) * psi[j-k] for k in range(1,j+1)])
        psi.append(psi_j)

    for j in range(max(p,q+1), 2*p+2*q):
        psi_j = sum([phi_coeff(k) * psi[j-k] for k in range(1,j+1)])
        psi.append(psi_j)

    gamma_symmetry_matrix = np.zeros((center, 2*center+1))
    gamma_symmetry_vector = np.zeros((center,))
    for j in range(1, max(p,q+1) + 1):
        gamma_symmetry_matrix[j-1,center+j] = 1
        gamma_symmetry_matrix[j-1,center-j] = -1
    
    gamma_boundary_matrix = np.zeros((center+1, 2*center+1))
    gamma_boundary_vector = np.zeros((center+1,))
    for k in range(0 , center+1):
        for j in range(0,p+1):
            gamma_boundary_matrix[k,center+k-j] = 1 if j == 0 else -phi_coeff(j)
        gamma_boundary_vector[k] = sum([theta_coeff(j)*psi[j-k] for j in range(k,q+1)])



    gamma_solution = np.linalg.solve(np.vstack([gamma_symmetry_matrix, gamma_boundary_matrix]),  np.hstack([gamma_symmetry_vector, gamma_boundary_vector]))[center:]

    for k in range(center+1,N):
        gamma_k = sum([phi_coeff(j) * gamma_solution[k-j] for j in range(1,p+1)])
        gamma_solution = np.append(gamma_solution, gamma_k)
    return gamma_solution

In [9]:
def innovations_algorithm(phi,theta,sigma,N = N):    
    gamma_solution = get_autocovariance(phi,theta)
    gamma = lambda h: gamma_solution[abs(h)]
    def kappa(i,j):
        if 1 <= min(i, j) and max(i, j) <= m:
            return sigma**(-2) * gamma(i - j)
        if min(i, j) <= m < max(i, j) <= 2 * m:
            return sigma**(-2) * (gamma(i - j) - sum([phi_coeff(r) * gamma(r - abs(i - j)) for r in range(1, p + 1)]))
        if min(i, j) > m:
            return sum([theta_coeff(r) * theta_coeff(r + abs(i - j)) for r in range(0, q + 1)])
        else:
            return 0

    kappa_matrix = np.array([[kappa(i,j) for i in range(0,N+3)]for j in range(0,N+3)]).astype(float)

    v_0 = kappa_matrix[1,1]
    v = np.array([v_0])
    O = np.empty((N+1, N+1))
    for n in range(1,N+1):
        for k in range(0,n):
            O[n,n-k] = v[k]**(-1) * (kappa_matrix[n+1,k+1] - sum([ O[k,k-j]*O[n,n-j]*v[j] for j in range(0,k) ]))
        v_n = kappa_matrix[n+1,n+1] - sum([ O[n,n-j]**2 * v[j] for j in range(0,n)])
        v = np.append(v,v_n)

    return O, v/sigma**2

In [10]:
def prediction(phi,theta,sigma,X):
    O, r = innovations_algorithm(phi,theta,sigma,N=len(X))
    Xhat = np.array([0])
    for n in range(0, len(X)):
        if 0 <= n < m:
            Xhat_n = sum([O[n,j] * ( X[n+1-j] - Xhat[n+1-j] ) for j in range(1, n+1)])
            Xhat = np.append(Xhat, Xhat_n)
        elif n >= m:
            Xhat_n = sum([phi_coeff(i)*X[n+1-i] for i in range(1,p+1)]) + sum([O[n,j] * ( X[n+1-j] - Xhat[n+1-j] ) for j in range(1, q+1)])
            Xhat = np.append(Xhat, Xhat_n)
    return Xhat


def h_step_prediction(phi,theta,sigma,X,h):
    n = len(X)-1

    O, r = innovations_algorithm(phi,theta,sigma,N=len(X)+h)
    Xhat = prediction(phi,theta,sigma,X)

    P_nX = np.append(X, Xhat[-1])
    for k in range(n+2, n+h+1):
        if k <= m:
            P_nX_k = sum([phi_coeff(i)*P_nX[k-i] for i in range(1,p+1)]) + sum([O[k-1,j] * ( X[k-j] - Xhat[k-n-j] ) for j in range(k-n, q+1)])
            P_nX = np.append(P_nX, P_nX_k)
        else:
            P_nX_k = sum([phi_coeff(i)*P_nX[k-i] for i in range(1,p+1)]) + sum([O[k-1,j] * ( X[k-j] - Xhat[k-j] ) for j in range(k-n, q+1)])
            P_nX = np.append(P_nX, P_nX_k)
    return P_nX




In [11]:
O, r = innovations_algorithm(phi,theta,sigma,N=len(X)+30)

In [12]:
h_step_prediction(phi,theta,sigma,X,3)[11:]

array([0.258854112159841, 0.0950054112159841, 0.0405630345807793],
      dtype=object)

In [13]:

partial_fraction_expansion = sp.apart(1/phi, full=True).nsimplify(tolerance=1e-10)
display(Math("\\chi(z) = \\phi(z)^{-1} =" + sp.latex(partial_fraction_expansion)))
chi_n = 0
polynomials = []
for partial_fraction in partial_fraction_expansion.args:
    if partial_fraction.is_polynomial() :
        polynomials.append(partial_fraction)
    else:
        poly = sp.Poly(partial_fraction**(-1), z)
        coeffs = (poly).coeffs()
        degree = sp.degree(poly, z)
        a = coeffs[-1]**(-1)
        r = sp.root((a*(-1)**degree) * coeffs[0],degree)
        chi_n += a*sp.binomial(n,degree-1)*r**n

chi = sp.lambdify(n, chi_n)
display(Math("\\chi_n =" + sp.latex(chi_n.simplify())))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [45]:
chi_coeff = lambda k : chi_list[k] if k >= 0 else 0
chi_list = [1]
for j in range(1,2*N):
    chi_list.append(sum([phi_coeff(k)*chi_coeff(j-k) for k in range(1,min(p,j)+1)]))

chi_list[:10], [chi(i) for i in range(10)]

([1,
  0.100000000000000,
  0.130000000000000,
  0.0250000000000000,
  0.0181000000000000,
  0.00481000000000000,
  0.00265300000000000,
  0.000842500000000000,
  0.000402610000000000,
  0.000141361000000000],
 [1.0,
  0.1,
  0.13,
  0.024999999999999998,
  0.018099999999999998,
  0.00481,
  0.002653,
  0.0008424999999999999,
  0.00040260999999999997,
  0.000141361])

In [14]:
n = len(X)-1
O, R = innovations_algorithm(phi,theta,sigma,N=N+2)
v = sigma**2 * R
mean_square_error = lambda h: sum([
    (sum([ chi(r)*O[n+h-r-1,h-j+r] for r in range(0,j+1)]))**2 * v[n+h-j-1] for j in range(0,h)])

mean_square_error(1), mean_square_error(2), mean_square_error(3) # 2.96 4.81

(0.4898512690589256, 0.48995174094140187, 0.49000096869989385)

In [15]:
# h = 2 j = 0
h = 2
theta_coeff = lambda k: theta.coeffs()[-1-k] if 0< k <= q else 1 if k == 0 else 0
sigma**2 * sum([sum([chi(r) * theta_coeff(j-r) for r in range(0,j+1)])**2 for j in range(0,h)])

1.36000000000000

In [39]:
mu_n = h_step_prediction(phi,theta,sigma,X,3)[11:].astype(float)
sigma_n = np.sqrt(np.array([mean_square_error(1), mean_square_error(2), mean_square_error(3)])).astype(float)
import scipy.stats as stats
alpha = 0.05

-stats.norm.ppf(np.array([alpha/2,]), loc =0, scale =sigma_n) + mu_n

array([1.63062067, 1.46691264, 1.41253918])